<a href="https://colab.research.google.com/github/sunshuofeng/AIssf/blob/master/REID_TEST1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#加载kaggle文件

In [1]:
!pip install kaggle

In [2]:
import json
token = {"username":"ssfailearning","key":"bccedfffce653f07d5acaf4409341c47"}
with open('/content/kaggle.json', 'w') as file:
  json.dump(token, file)

In [3]:
!mkdir -p ~/.kaggle


In [4]:
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v /content

- path is now set to: /content


In [5]:
!kaggle datasets download -d rayiooo/reid_market-1501

reid_market-1501.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
!pip install geffnet
!pip install git+https://github.com/pabloppp/pytorch-tools -U
!pip install thop 

  Cloning https://github.com/pabloppp/pytorch-tools to /tmp/pip-req-build-llusxk_e
  Running command git clone -q https://github.com/pabloppp/pytorch-tools /tmp/pip-req-build-llusxk_e
  Created wheel for torchtools: filename=torchtools-0.2.6-cp37-none-any.whl size=21149 sha256=9bd03eee353c6eb3e94086858e5c40f7d3af126d5c38487d4ea5691e81e1412f
  Stored in directory: /tmp/pip-ephem-wheel-cache-ljzif4_o/wheels/53/7a/b0/86ffb126404564b151069f4d8d9f43dbfe3f17bfc0b898b45f
Successfully built torchtools
  Found existing installation: torchtools 0.2.6
    Uninstalling torchtools-0.2.6:
      Successfully uninstalled torchtools-0.2.6


# <font color=red>***配置文件***</font>

In [7]:
import random
import numpy as np
import torch
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.benchmark=True
torch.manual_seed(42)
torch.cuda.manual_seed(42)       
cfg={
    'train_size':[384,128],
    'test_size':[384,128],
    'padding':10,
    'mean':[0.485, 0.456, 0.406],
    'std':[0.229, 0.224, 0.225],

    'Multi_Data':False,
    'num_workers':8,
    'SAMPLER':'',

    'train_bs':48,
    'test_bs':48,

    'train_K_instances':12,
     
    'epochs':70,
    'ckpt':'reid_test1.pt',
    'logpt':'test1.log',

    'rerank':False,
   
    'lr':3e-4,
    'momentum':0.9,
    'weight_decay':5e-4,
     
    'margin':0.3,
    'steps':[35,55],
    'warmup_iters':10,
    'warmup_factors':1/3,
    'gamma':0.1,
    'model':'baseline',
    'backbone':'resnet50',
    'last_stride':(1,1),
    'optimizer':'adam',
    'scheduler':'warmup',
    'grad_l2':False,
     
    'loss':[('cross',1.0),('cirlce',1.0)],

    'two_stage':False, 

    'amp':True,
    'ema':False,
   
    
}

# <font color=red>***读取数据***</font>

In [8]:
# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

##<font color=blue>***取market1501***</font>

In [9]:
import os
import zipfile
# os.makedirs('datasets')
zip_file = zipfile.ZipFile('/content/drive/MyDrive/dataset/REID/reid1.zip')
for names in zip_file.namelist():
        zip_file.extract(names,'datasets')
zip_file.close()

In [10]:
import numpy as np
import pandas as pd
from torch.cuda.amp import GradScaler,autocast
import torch


##<font color=blue>***数据增强***</font>

###<font color=green>***图像***</font>

#### <font color=yellow>***随机擦除***</font>

In [11]:
import math
import random


class RandomErasing(object):
    """ Randomly selects a rectangle region in an image and erases its pixels.
        'Random Erasing Data Augmentation' by Zhong et al.
        See https://arxiv.org/pdf/1708.04896.pdf
    Args:
         probability: The probability that the Random Erasing operation will be performed.
         sl: Minimum proportion of erased area against input image.
         sh: Maximum proportion of erased area against input image.
         r1: Minimum aspect ratio of erased area.
         mean: Erasing value.
    """

    def __init__(self, probability=0.5, sl=0.02, sh=0.4, r1=0.3, mean=(0.4914, 0.4822, 0.4465)):
        self.probability = probability
        self.mean = mean
        self.sl = sl
        self.sh = sh
        self.r1 = r1

    def __call__(self, img):

        if random.uniform(0, 1) >= self.probability:
            return img

        for attempt in range(100):
            area = img.size()[1] * img.size()[2]

            target_area = random.uniform(self.sl, self.sh) * area
            aspect_ratio = random.uniform(self.r1, 1 / self.r1)

            h = int(round(math.sqrt(target_area * aspect_ratio)))
            w = int(round(math.sqrt(target_area / aspect_ratio)))

            if w < img.size()[2] and h < img.size()[1]:
                x1 = random.randint(0, img.size()[1] - h)
                y1 = random.randint(0, img.size()[2] - w)
                if img.size()[0] == 3:
                    img[0, x1:x1 + h, y1:y1 + w] = self.mean[0]
                    img[1, x1:x1 + h, y1:y1 + w] = self.mean[1]
                    img[2, x1:x1 + h, y1:y1 + w] = self.mean[2]
                else:
                    img[0, x1:x1 + h, y1:y1 + w] = self.mean[0]
                return img

        return img

#### 灰度+sketch

In [12]:

import math
from PIL import Image  #PIL=python image library
import random
import  numpy as np
import random
import cv2



########################### this code is for Local Grayscale Patch Replacement(LGPR)  #################################
class LGPR(object):

    def __init__(self, probability=0.2, sl=0.02, sh=0.4, r1=0.3):
        self.probability = probability
        self.sl = sl
        self.sh = sh
        self.r1 = r1

    def __call__(self, img):

        new = img.convert("L")
        np_img = np.array(new, dtype=np.uint8)
        img_gray = np.dstack([np_img, np_img, np_img])

        if random.uniform(0, 1) >= self.probability:
            return img

        for attempt in range(100):
            area = img.size[0] * img.size[1]
            target_area = random.uniform(self.sl, self.sh) * area
            aspect_ratio = random.uniform(self.r1, 1 / self.r1)

            h = int(round(math.sqrt(target_area * aspect_ratio)))
            w = int(round(math.sqrt(target_area / aspect_ratio)))

            if w < img.size[1] and h < img.size[0]:
                x1 = random.randint(0, img.size[0] - h)
                y1 = random.randint(0, img.size[1] - w)
                img = np.asarray(img).astype('float')

                img[y1:y1 + h, x1:x1 + w, 0] = img_gray[y1:y1 + h, x1:x1 + w, 0]
                img[y1:y1 + h, x1:x1 + w, 1] = img_gray[y1:y1 + h, x1:x1 + w, 1]
                img[y1:y1 + h, x1:x1 + w, 2] = img_gray[y1:y1 + h, x1:x1 + w, 2]

                img = Image.fromarray(img.astype('uint8'))

                return img

        return img
#######################################################################################################################
################################ this code is for Multi-Modal Defense  ################################################

def toSketch(img):  # Convert visible  image to sketch image
    img_np = np.asarray(img)
    img_inv = 255 - img_np
    img_blur = cv2.GaussianBlur(img_inv, ksize=(27, 27), sigmaX=0, sigmaY=0)
    img_blend = cv2.divide(img_np, 255 - img_blur, scale=256)
    img_blend = Image.fromarray(img_blend)
    return img_blend

"""
Randomly select several channels of visible image (R, G, B), gray image (gray), and sketch image (sketch) 
to fuse them into a new 3-channel image.
"""
def random_choose(r, g, b, gray_or_sketch):
    p = [r, g, b, gray_or_sketch, gray_or_sketch]
    idx = [0, 1, 2, 3, 4]
    random.shuffle(idx)
    return Image.merge('RGB', [p[idx[0]], p[idx[1]], p[idx[2]]])


# 10(%Grayscale) 5%(Grayscale-RGB) 5%(Sketch-RGB)
class Fuse_RGB_Gray_Sketch(object):
    def __init__(self,G=0.1,G_rgb = 0.05,S_rgb =0.05):
        self.G = G
        self.G_rgb = G_rgb
        self.S_rgb = S_rgb

    def __call__(self, img):
        r, g, b = img.split()
        gray = img.convert('L') #convert visible  image to grayscale images
        p = random.random()
        if p < self.G: #just Grayscale
            return Image.merge('RGB', [gray, gray, gray])

        elif p < self.G + self.G_rgb: #fuse Grayscale-RGB
            img2 = random_choose(r, g, b, gray)
            return img2

        elif p < self.G + self.G_rgb + self.S_rgb: #fuse Sketch-RGB
            sketch = toSketch(gray)
            img3 = random_choose(r, g, b, sketch)
            return img3
        else:
            return img

####<font color=yellow>***其余数据增强（torchvision transformss）***</font>



In [13]:

import torchvision.transforms as T



def build_transforms(cfg, training=True):
    normalize_transform = T.Normalize(mean=cfg['mean'], std=cfg['std'])
    if training:
        transform = T.Compose([
            T.Resize(cfg['train_size']),
            T.RandomHorizontalFlip(p=0.5),
            T.Pad(cfg['padding']),
            T.RandomCrop(cfg['train_size']),
        
            T.ToTensor(),
            normalize_transform,
            RandomErasing(probability=0.5, mean=cfg['mean'])
        ])
    else:
        transform = T.Compose([
            T.Resize(cfg['test_size']),
            T.ToTensor(),
            normalize_transform
        ])

    return transform



In [14]:
def build_change_transforms(cfg, training=True):
    normalize_transform = T.Normalize(mean=cfg['mean'], std=cfg['std'])
    if training:
        transform = T.Compose([
            T.Resize(cfg['train_size']),
            T.ColorJitter(),
            T.RandomHorizontalFlip(p=0.5),
            T.Pad(cfg['padding']),
            T.RandomCrop(cfg['train_size']),
            T.RandomGrayscale(p=0.1),
            T.ToTensor(),
            normalize_transform,
            RandomErasing(probability=0.5, mean=cfg['mean']),
            
        ])
    else:
        transform = T.Compose([
            T.Resize(cfg['test_size']),
            T.ToTensor(),
            normalize_transform
        ])

    return transform


##<font color=blue>***sampler***</font>

<font color=red>抽取N个pid(即N个人)，每个人抽取K张图片，目的是为了能确保至少有三元组，**锚，正样本，负样本**</font>


###<font color=green>***图像***</font>

In [15]:

import copy
import random
import torch
from collections import defaultdict

import numpy as np
from torch.utils.data.sampler import Sampler

class RandomIdentitySampler(Sampler):
    """
    Randomly sample N identities, then for each identity,
    randomly sample K instances, therefore batch size is N*K.
    Args:
    - data_source (list): list of (img_path, pid, camid).
    - num_instances (int): number of instances per identity in a batch.
    - batch_size (int): number of examples in a batch.
    """

    def __init__(self, data_source, batch_size, num_instances):
        self.data_source = data_source
        self.batch_size = batch_size
        self.num_instances = num_instances
        self.num_pids_per_batch = self.batch_size // self.num_instances
        self.index_dic = defaultdict(list)
        for index, (_, pid, _) in enumerate(self.data_source):
            self.index_dic[pid].append(index)
        self.pids = list(self.index_dic.keys())

        # estimate number of examples in an epoch
        self.length = 0
        for pid in self.pids:
            idxs = self.index_dic[pid]
            num = len(idxs)
            if num < self.num_instances:
                num = self.num_instances
            self.length += num - num % self.num_instances

    def __iter__(self):
        batch_idxs_dict = defaultdict(list)

        for pid in self.pids:
            idxs = copy.deepcopy(self.index_dic[pid])
            if len(idxs) < self.num_instances:
                idxs = np.random.choice(idxs, size=self.num_instances, replace=True)
            random.shuffle(idxs)
            batch_idxs = []
            for idx in idxs:
                batch_idxs.append(idx)
                if len(batch_idxs) == self.num_instances:
                    batch_idxs_dict[pid].append(batch_idxs)
                    batch_idxs = []

        avai_pids = copy.deepcopy(self.pids)
        final_idxs = []

        while len(avai_pids) >= self.num_pids_per_batch:
            selected_pids = random.sample(avai_pids, self.num_pids_per_batch)
            for pid in selected_pids:
                batch_idxs = batch_idxs_dict[pid].pop(0)
                final_idxs.extend(batch_idxs)
                if len(batch_idxs_dict[pid]) == 0:
                    avai_pids.remove(pid)

        self.length = len(final_idxs)
        return iter(final_idxs)

    def __len__(self):
        return self.length


In [16]:
class DateRandomIdentitySampler(Sampler):
    """
    Randomly sample N identities, then for each identity,
    randomly sample K instances, therefore batch size is N*K.
    Args:
    - data_source (list): list of (img_path, pid, camid).
    - num_instances (int): number of instances per identity in a batch.
    - batch_size (int): number of examples in a batch.
    """

    def __init__(self, data_source, batch_size, num_instances):
        self.data_source = data_source
        self.batch_size = batch_size
        self.num_instances = num_instances
        self.num_pids_per_batch = self.batch_size // self.num_instances
        self.index_dic = defaultdict(list)
        for index, (_, pid, _,_) in enumerate(self.data_source):
            self.index_dic[pid].append(index)
        self.pids = list(self.index_dic.keys())

        # estimate number of examples in an epoch
        self.length = 0
        for pid in self.pids:
            idxs = self.index_dic[pid]
            num = len(idxs)
            if num < self.num_instances:
                num = self.num_instances
            self.length += num - num % self.num_instances

    def __iter__(self):
        batch_idxs_dict = defaultdict(list)

        for pid in self.pids:
            idxs = copy.deepcopy(self.index_dic[pid])
            if len(idxs) < self.num_instances:
                idxs = np.random.choice(idxs, size=self.num_instances, replace=True)
            random.shuffle(idxs)
            batch_idxs = []
            for idx in idxs:
                batch_idxs.append(idx)
                if len(batch_idxs) == self.num_instances:
                    batch_idxs_dict[pid].append(batch_idxs)
                    batch_idxs = []

        avai_pids = copy.deepcopy(self.pids)
        final_idxs = []

        while len(avai_pids) >= self.num_pids_per_batch:
            selected_pids = random.sample(avai_pids, self.num_pids_per_batch)
            for pid in selected_pids:
                batch_idxs = batch_idxs_dict[pid].pop(0)
                final_idxs.extend(batch_idxs)
                if len(batch_idxs_dict[pid]) == 0:
                    avai_pids.remove(pid)

        self.length = len(final_idxs)
        return iter(final_idxs)

    def __len__(self):
        return self.length

##<font color=blue>***建立数据集***</font>

###<font color=green>***图像***</font>

####<font color=yellow>***图像读取数据集（market1501）***</font>

In [17]:
import glob
import re
import os.path as osp
class Market1501():
    def __init__(self, root='/content/datasets', verbose=True, **kwargs):
        super(Market1501, self).__init__()
        self.dataset_dir = root
        self.train_dir = osp.join(self.dataset_dir, 'bounding_box_train')
        self.query_dir = osp.join(self.dataset_dir, 'query')
        self.gallery_dir = osp.join(self.dataset_dir, 'bounding_box_test')
        self._check_before_run()

        train = self._process_dir(self.train_dir, relabel=True)
        query = self._process_dir(self.query_dir, relabel=False)
        gallery = self._process_dir(self.gallery_dir, relabel=False)

        if verbose:
            print("=> Market1501 loaded")
            self.print_dataset_statistics(train, query, gallery)

        self.train = train
        self.query = query
        self.gallery = gallery

        self.num_train_pids, self.num_train_imgs, self.num_train_cams = self.get_imagedata_info(self.train)
        self.num_query_pids, self.num_query_imgs, self.num_query_cams = self.get_imagedata_info(self.query)
        self.num_gallery_pids, self.num_gallery_imgs, self.num_gallery_cams = self.get_imagedata_info(self.gallery)

    def print_dataset_statistics(self, train, query, gallery):
        num_train_pids, num_train_imgs, num_train_cams = self.get_imagedata_info(train)
        num_query_pids, num_query_imgs, num_query_cams = self.get_imagedata_info(query)
        num_gallery_pids, num_gallery_imgs, num_gallery_cams = self.get_imagedata_info(gallery)

        print("Dataset statistics:")
        print("  ----------------------------------------")
        print("  subset   | # ids | # images | # cameras")
        print("  ----------------------------------------")
        print("  train    | {:5d} | {:8d} | {:9d}".format(num_train_pids, num_train_imgs, num_train_cams))
        print("  query    | {:5d} | {:8d} | {:9d}".format(num_query_pids, num_query_imgs, num_query_cams))
        print("  gallery  | {:5d} | {:8d} | {:9d}".format(num_gallery_pids, num_gallery_imgs, num_gallery_cams))
        print("  ----------------------------------------")
    
    def get_imagedata_info(self, data):
        pids, cams = [], []
        for _, pid, camid in data:
            pids += [pid]
            cams += [camid]
        pids = set(pids)
        cams = set(cams)
        num_pids = len(pids)
        num_cams = len(cams)
        num_imgs = len(data)
        return num_pids, num_imgs, num_cams
    

    def _check_before_run(self):
        """Check if all files are available before going deeper"""
        if not osp.exists(self.dataset_dir):
            raise RuntimeError("'{}' is not available".format(self.dataset_dir))
        if not osp.exists(self.train_dir):
            raise RuntimeError("'{}' is not available".format(self.train_dir))
        if not osp.exists(self.query_dir):
            raise RuntimeError("'{}' is not available".format(self.query_dir))
        if not osp.exists(self.gallery_dir):
            raise RuntimeError("'{}' is not available".format(self.gallery_dir))

    
    ##最终返回的是每张图片的路径，pid，camid
    def _process_dir(self, dir_path, relabel=False):
        img_paths = glob.glob(osp.join(dir_path, '*.jpg'))

        ##匹配文件名中的有用信息，0001_c1代表pid=1，cid=1，
        ##至于为什么是[-\d]+,是因为有-1_c1这种的存在，也应该匹配上
        pattern = re.compile(r'([-\d]+)_c(\d)')

        pid_container = set()
        for img_path in img_paths:
            pid, _ = map(int, pattern.search(img_path).groups())
            if pid == -1: continue  # junk images are just ignored
            pid_container.add(pid)
        pid2label = {pid: label for label, pid in enumerate(pid_container)}

        dataset = []
        for img_path in img_paths:
            pid, camid = map(int, pattern.search(img_path).groups())
            if pid == -1: continue  # junk images are just ignored
            assert 0 <= pid <= 1501  # pid == 0 means background
            assert 1 <= camid <= 6
            camid -= 1  # index starts from 0
            if relabel: pid = pid2label[pid]
            dataset.append((img_path, pid, camid))

        return dataset


import torch.utils.data as Data
from PIL import Image


class ImageDataset(Data.Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        img_path, pid, camid = self.dataset[index]
        img = Image.open(img_path)
        if self.transform is not None:
            img = self.transform(img)
        return img, pid, camid, img_path



In [18]:
from fastai.vision import get_image_files
import torch.utils.data as Data
import json


class PDataset:
  def __init__(self,root):
    self.root=root
    self.train_dir=osp.join(self.root,'train')
    self.query_dir=osp.join(self.root,'query')
    self.gallery_dir=osp.join(self.root,'gallery')
    
    train=self._process_dir(self.train_dir)
    query=self._process_dir(self.query_dir)
    gallery=self._process_dir(self.gallery_dir)
    self.train=train
    self.query=query
    self.gallery=gallery
    self.num_train_pids, self.num_train_imgs, self.num_train_cams = self.get_imagedata_info(self.train)
    self.num_query_pids, self.num_query_imgs, self.num_query_cams = self.get_imagedata_info(self.query)
    self.num_gallery_pids, self.num_gallery_imgs, self.num_gallery_cams = self.get_imagedata_info(self.gallery)

  
  def get_imagedata_info(self,data):
    pids,cams=[],[]
    for _,pid,camid,_ in data:
      pids+=[pid]
      cams+=[camid]
    pids=set(pids)
    cams=set(cams)
    num_pids=len(pids)
    num_cams=len(cams)
    num_imgs=len(data)
    return num_pids,num_imgs,num_cams

  def _process_dir(self,dir_path):
    image_names=get_image_files(dir_path)
    dataset=[]
    pid_set=set()
    for name in image_names:
      name=str(name)
      path=name.split('/')[-1]
      pid,_,date,_=path.split('_')
      pid_set.add(pid)
    pid2label={pid: label for label, pid in enumerate(pid_set)}
    for name in image_names:
      name=str(name)
      path=name.split('/')[-1]
      pid,_,date,_=path.split('_')
      pid=pid2label[pid]
      cam=image_cams[path]
      date=date_ids[date]
      dataset.append((name,int(pid),int(cam),int(date)))
    return dataset

class ImageDataset(Data.Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        img_path, pid, camid,date = self.dataset[index]
        img = Image.open(img_path)
        if self.transform is not None:
            img = self.transform(img)
        return img, pid, camid,img_path,date


###<font color=green>***建立dataloader***</font>

In [19]:

###对于训练的时候我们不需要关心摄像头的问题，我们只需要让模型能认出图片里的人就行
def train_collate_fn(batch):
    imgs, pids, _, _, = zip(*batch)
    pids = torch.tensor(pids, dtype=torch.int64)
    return torch.stack(imgs, dim=0), pids

###对于验证集而言，为了提高验证的真实性，我们应该防止同一摄像头的图片进入验证（同一摄像头相当于数据泄露）
def val_collate_fn(batch):
    imgs, pids, camids, _ = zip(*batch)
    return torch.stack(imgs, dim=0), pids, camids



def make_data_loader(datasets,cfg):
    train_transform=build_transforms(cfg,training=True)
    test_transform=build_transforms(cfg,training=False)
    if cfg['Multi_Data']:
        data=datasets[0]
        for dataset in datasets[1:]:
            data.train.extend(dataset.train)
            data.query.extend(dataset.query)
            data.gallery.extend(dataset.gallery)

            data.num_train_pids+=dataset.num_train_pids
    else:
        data=datasets

    num_classes=data.num_train_pids
    train_set=ImageDataset(data.train,train_transform)

    if cfg['SAMPLER']=='softmax':
        train_loader=Data.DataLoader(train_set,batch_size=cfg['train_bs'],
                                     shuffle=True,collate_fn=train_collate_fn)
    else:
        train_loader=Data.DataLoader(train_set,batch_size=cfg['train_bs'],
                                     sampler=RandomIdentitySampler(data.train,
                                    cfg['train_bs'],cfg['train_K_instances']),
                                    num_workers=cfg['num_workers'],
                                    collate_fn=train_collate_fn)
    
    val_set = ImageDataset(data.query + data.gallery, test_transform)
    val_loader = Data.DataLoader(
        val_set, batch_size=cfg['test_bs'], shuffle=False, num_workers=cfg['num_workers'],
        collate_fn=val_collate_fn
    )
    return train_loader, val_loader, len(data.query), num_classes



In [20]:
def train_collate_fn1(batch):
    imgs, pids, _, _,_ = zip(*batch)
    pids = torch.tensor(pids, dtype=torch.int64)
    return torch.stack(imgs, dim=0), pids

###对于验证集而言，为了提高验证的真实性，我们应该防止同一摄像头的图片进入验证（同一摄像头相当于数据泄露）
def val_collate_fn1(batch):
    imgs, pids, camids, _,dates = zip(*batch)
    return torch.stack(imgs, dim=0), pids, camids,dates



def make_date_data_loader(datasets,cfg):
    train_transform=build_change_transforms(cfg,training=True)
    test_transform=build_change_transforms(cfg,training=False)
    if cfg['Multi_Data']:
        data=datasets[0]
        for dataset in datasets[1:]:
            data.train.extend(dataset.train)
            data.query.extend(dataset.query)
            data.gallery.extend(dataset.gallery)

            data.num_train_pids+=dataset.num_train_pids
    else:
        data=datasets

    num_classes=data.num_train_pids
    train_set=ImageDataset(data.train,train_transform)

    if cfg['SAMPLER']=='softmax':
        train_loader=Data.DataLoader(train_set,batch_size=cfg['train_bs'],
                                     shuffle=True,collate_fn=train_collate_fn1)
    else:
        train_loader=Data.DataLoader(train_set,batch_size=cfg['train_bs'],
                                     sampler=DateRandomIdentitySampler(data.train,
                                    cfg['train_bs'],cfg['train_K_instances']),
                                    num_workers=cfg['num_workers'],
                                    collate_fn=train_collate_fn1)
    
    val_set = ImageDataset(data.query + data.gallery, test_transform)
    val_loader = Data.DataLoader(
        val_set, batch_size=cfg['test_bs'], shuffle=False, num_workers=cfg['num_workers'],
        collate_fn=val_collate_fn1
    )
    return train_loader, val_loader, len(data.query), num_classes


#<font color=red>***模型建立***</font>

##<font color=blue>***图像***</font>

###<font color=green>***backbone***</font>

####<font color=yellow>***resnet50***</font>

In [21]:
from torchvision.models import resnet50
def ResNet50(cfg):
  model=resnet50(True)
  stride=cfg['last_stride']
  model.layer4[0].downsample[0].stride=stride
  model.layer4[0].conv2.stride=stride
  base=nn.Sequential(
            model.conv1,
            model.bn1,
            model.maxpool,
            model.layer1,
            model.layer2,
            model.layer3,
            model.layer4
        )
  return base,2048

####<font color=yellow>***resnet101***</font>

In [22]:
from torchvision.models import resnet101
def ResNet101(cfg):
  model=resnet101(True)
  stride=cfg['last_stride']
  model.layer4[0].downsample[0].stride=stride
  model.layer4[0].conv2.stride=stride
  base=nn.Sequential(
            model.conv1,
            model.bn1,
            model.maxpool,
            model.layer1,
            model.layer2,
            model.layer3,
            model.layer4
        )
  return base,2048

####<font color=yellow>***effnet_b3***</font>

In [23]:
from geffnet import efficientnet_b3
def Eff_b3(cfg):
  model=efficientnet_b3(True)
  base=nn.Sequential(
    model.conv_stem,
    model.bn1,
    model.act1,
    model.blocks,
    model.conv_head,
    model.bn2,
    model.act2
  )
  return base,1536

####<font color=yellow>***effnet_b5***

In [24]:
from geffnet import efficientnet_b5
def Eff_b5(cfg):
  model=efficientnet_b5(True)
  base=nn.Sequential(
    model.conv_stem,
    model.bn1,
    model.act1,
    model.blocks,
    model.conv_head,
    model.bn2,
    model.act2
  )
  return base,2048

####<font color=yellow>***eff_b6***

In [25]:
from geffnet import tf_efficientnet_b6_ap
def Eff_b6(cfg):
  model=tf_efficientnet_b6_ap(True)
  base=nn.Sequential(
    model.conv_stem,
    model.bn1,
    model.act1,
    model.blocks,
    model.conv_head,
    model.bn2,
    model.act2
  )
  return base,2304

####<font color=yellow>***eff_b7***</font>

In [26]:
from geffnet import tf_efficientnet_b7_ap
def Eff_b7(cfg):
  model=tf_efficientnet_b7_ap(True)
  base=nn.Sequential(
    model.conv_stem,
    model.bn1,
    model.act1,
    model.blocks,
    model.conv_head,
    model.bn2,
    model.act2
  )
  return base,2560

###<font color=green>***strong-baseline***</font>

In [27]:
from torchvision.models import resnet50
import torch.nn as nn
def weights_init_kaiming(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.kaiming_normal_(m.weight, a=0, mode='fan_out')
        nn.init.constant_(m.bias, 0.0)
    elif classname.find('Conv') != -1:
        nn.init.kaiming_normal_(m.weight, a=0, mode='fan_in')
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)
    elif classname.find('BatchNorm') != -1:
        if m.affine:
            nn.init.constant_(m.weight, 1.0)
            nn.init.constant_(m.bias, 0.0)


def weights_init_classifier(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.normal_(m.weight, std=0.001)
        if m.bias:
            nn.init.constant_(m.bias, 0.0)
class Baseline(nn.Module):
   
    def __init__(self,num_classes,cfg,neck='bnneck', neck_feat='after'):
        super(Baseline, self).__init__()
        if cfg['backbone']=='resnet50':
          self.base,in_planes=ResNet50(cfg)
        if cfg['backbone']=='resnet101':
          self.base,in_planes=ResNet101(cfg) 
        if cfg['backbone']=='eff_b3':
          self.base,in_planes=Eff_b3(cfg)  
        if cfg['backbone']=='eff_b5':
          self.base,in_planes=Eff_b5(cfg)
        if cfg['backbone']=='eff_b7':
          self.base,in_planes=Eff_b7(cfg)
        if cfg['backbone']=='eff_b6':
          self.base,in_planes=Eff_b6(cfg)   


        self.in_planes=in_planes
        self.gap = nn.AdaptiveAvgPool2d(1)
        # self.gap = nn.AdaptiveMaxPool2d(1)
        self.num_classes = num_classes
        self.neck = neck
        self.neck_feat = neck_feat

        if self.neck == 'no':
            self.classifier = nn.Linear(self.in_planes, self.num_classes)
            # self.classifier = nn.Linear(self.in_planes, self.num_classes, bias=False)     # new add by luo
            # self.classifier.apply(weights_init_classifier)  # new add by luo
        elif self.neck == 'bnneck':
            self.bottleneck = nn.BatchNorm1d(self.in_planes)
            self.bottleneck.bias.requires_grad_(False)  # no shift
            self.classifier = nn.Linear(self.in_planes, self.num_classes, bias=False)

            self.bottleneck.apply(weights_init_kaiming)
            self.classifier.apply(weights_init_classifier)

    def forward(self, x):

        global_feat = self.gap(self.base(x))  # (b, 2048, 1, 1)
        global_feat = global_feat.view(global_feat.shape[0], -1)  # flatten to (bs, 2048)
        out={}
        if self.neck == 'no':
            feat = global_feat
        elif self.neck == 'bnneck':
            feat = self.bottleneck(global_feat)  # normalize for angular softmax

        if self.training:
            cls_score = self.classifier(feat)
            out['logit']=cls_score
            out['feat']=global_feat
            return out  # global feature for triplet loss
        else:
            if self.neck_feat == 'after':
                # print("Test with feature after BN")
                out['feat']=feat
                return out
            else:
                # print("Test with feature before BN")
                out['feat']=global_feat
                return out

    def load_param(self, trained_path):
        param_dict = torch.load(trained_path)
        for i in param_dict:
            if 'classifier' in i:
                continue
            self.state_dict()[i].copy_(param_dict[i])

###<font color=green>***全局注意力网络***

In [28]:

import torch
from torch import nn
from torch.nn import functional as F

import pdb

# ===================
#     RGA Module
# ===================

class RGA_Module(nn.Module):
	def __init__(self, in_channel, in_spatial, use_spatial=True, use_channel=True, \
		cha_ratio=8, spa_ratio=8, down_ratio=8):
		super(RGA_Module, self).__init__()

		self.in_channel = in_channel
		self.in_spatial = in_spatial
		
		self.use_spatial = use_spatial
		self.use_channel = use_channel

		print ('Use_Spatial_Att: {};\tUse_Channel_Att: {}.'.format(self.use_spatial, self.use_channel))

		self.inter_channel = in_channel // cha_ratio
		self.inter_spatial = in_spatial // spa_ratio
		
		# Embedding functions for original features
		if self.use_spatial:
			self.gx_spatial = nn.Sequential(
				nn.Conv2d(in_channels=self.in_channel, out_channels=self.inter_channel,
						kernel_size=1, stride=1, padding=0, bias=False),
				nn.BatchNorm2d(self.inter_channel),
				nn.ReLU()
			)
		if self.use_channel:
			self.gx_channel = nn.Sequential(
				nn.Conv2d(in_channels=self.in_spatial, out_channels=self.inter_spatial,
						kernel_size=1, stride=1, padding=0, bias=False),
				nn.BatchNorm2d(self.inter_spatial),
				nn.ReLU()
			)
		
		# Embedding functions for relation features
		if self.use_spatial:
			self.gg_spatial = nn.Sequential(
				nn.Conv2d(in_channels=self.in_spatial * 2, out_channels=self.inter_spatial,
						kernel_size=1, stride=1, padding=0, bias=False),
				nn.BatchNorm2d(self.inter_spatial),
				nn.ReLU()
			)
		if self.use_channel:
			self.gg_channel = nn.Sequential(
				nn.Conv2d(in_channels=self.in_channel*2, out_channels=self.inter_channel,
						kernel_size=1, stride=1, padding=0, bias=False),
				nn.BatchNorm2d(self.inter_channel),
				nn.ReLU()
			)
		
		# Networks for learning attention weights
		if self.use_spatial:
			num_channel_s = 1 + self.inter_spatial
			self.W_spatial = nn.Sequential(
				nn.Conv2d(in_channels=num_channel_s, out_channels=num_channel_s//down_ratio,
						kernel_size=1, stride=1, padding=0, bias=False),
				nn.BatchNorm2d(num_channel_s//down_ratio),
				nn.ReLU(),
				nn.Conv2d(in_channels=num_channel_s//down_ratio, out_channels=1,
						kernel_size=1, stride=1, padding=0, bias=False),
				nn.BatchNorm2d(1)
			)
		if self.use_channel:	
			num_channel_c = 1 + self.inter_channel
			self.W_channel = nn.Sequential(
				nn.Conv2d(in_channels=num_channel_c, out_channels=num_channel_c//down_ratio,
						kernel_size=1, stride=1, padding=0, bias=False),
				nn.BatchNorm2d(num_channel_c//down_ratio),
				nn.ReLU(),
				nn.Conv2d(in_channels=num_channel_c//down_ratio, out_channels=1,
						kernel_size=1, stride=1, padding=0, bias=False),
				nn.BatchNorm2d(1)
			)

		# Embedding functions for modeling relations
		if self.use_spatial:
			self.theta_spatial = nn.Sequential(
				nn.Conv2d(in_channels=self.in_channel, out_channels=self.inter_channel,
								kernel_size=1, stride=1, padding=0, bias=False),
				nn.BatchNorm2d(self.inter_channel),
				nn.ReLU()
			)
			self.phi_spatial = nn.Sequential(
				nn.Conv2d(in_channels=self.in_channel, out_channels=self.inter_channel,
							kernel_size=1, stride=1, padding=0, bias=False),
				nn.BatchNorm2d(self.inter_channel),
				nn.ReLU()
			)
		if self.use_channel:
			self.theta_channel = nn.Sequential(
				nn.Conv2d(in_channels=self.in_spatial, out_channels=self.inter_spatial,
								kernel_size=1, stride=1, padding=0, bias=False),
				nn.BatchNorm2d(self.inter_spatial),
				nn.ReLU()
			)
			self.phi_channel = nn.Sequential(
				nn.Conv2d(in_channels=self.in_spatial, out_channels=self.inter_spatial,
							kernel_size=1, stride=1, padding=0, bias=False),
				nn.BatchNorm2d(self.inter_spatial),
				nn.ReLU()
			)
				
	def forward(self, x):
		b, c, h, w = x.size()
		
		if self.use_spatial:
			# spatial attention
			theta_xs = self.theta_spatial(x)	
			phi_xs = self.phi_spatial(x)
			theta_xs = theta_xs.view(b, self.inter_channel, -1)
			theta_xs = theta_xs.permute(0, 2, 1)
			phi_xs = phi_xs.view(b, self.inter_channel, -1)
			Gs = torch.matmul(theta_xs, phi_xs)
			Gs_in = Gs.permute(0, 2, 1).view(b, h*w, h, w)
			Gs_out = Gs.view(b, h*w, h, w)
			Gs_joint = torch.cat((Gs_in, Gs_out), 1)
			Gs_joint = self.gg_spatial(Gs_joint)
		
			g_xs = self.gx_spatial(x)
			g_xs = torch.mean(g_xs, dim=1, keepdim=True)
			ys = torch.cat((g_xs, Gs_joint), 1)

			W_ys = self.W_spatial(ys)
			if not self.use_channel:
				out = F.sigmoid(W_ys.expand_as(x)) * x
				return out
			else:
				x = F.sigmoid(W_ys.expand_as(x)) * x

		if self.use_channel:
			# channel attention
			xc = x.view(b, c, -1).permute(0, 2, 1).unsqueeze(-1)
			theta_xc = self.theta_channel(xc).squeeze(-1).permute(0, 2, 1)
			phi_xc = self.phi_channel(xc).squeeze(-1)
			Gc = torch.matmul(theta_xc, phi_xc)
			Gc_in = Gc.permute(0, 2, 1).unsqueeze(-1)
			Gc_out = Gc.unsqueeze(-1)
			Gc_joint = torch.cat((Gc_in, Gc_out), 1)
			Gc_joint = self.gg_channel(Gc_joint)

			g_xc = self.gx_channel(xc)
			g_xc = torch.mean(g_xc, dim=1, keepdim=True)
			yc = torch.cat((g_xc, Gc_joint), 1)

			W_yc = self.W_channel(yc).transpose(1, 2)
			out = F.sigmoid(W_yc) * x

			return out


def weights_init_kaiming(m):
	classname = m.__class__.__name__
	if classname.find('Linear') != -1:
		nn.init.kaiming_normal_(m.weight, a=0, mode='fan_out')
		nn.init.constant_(m.bias, 0.0)
	elif classname.find('Conv') != -1:
		nn.init.kaiming_normal_(m.weight, a=0, mode='fan_in')
		if m.bias is not None:
			nn.init.constant_(m.bias, 0.0)
	elif classname.find('BatchNorm') != -1:
		if m.affine:
			nn.init.normal_(m.weight, 1.0, 0.02)
			nn.init.constant_(m.bias, 0.0)


def weights_init_fc(m):
	classname = m.__class__.__name__
	if classname.find('Linear') != -1:
		nn.init.normal_(m.weight, std=0.001)
		nn.init.constant_(m.bias, 0.0)
	elif classname.find('BatchNorm') != -1:
		if m.affine:
			nn.init.normal_(m.weight, 1.0, 0.02)
			nn.init.constant_(m.bias, 0.0)


class Bottleneck(nn.Module):
	expansion = 4

	def __init__(self, in_channels, out_channels, stride=1, downsample=None):
		super(Bottleneck, self).__init__()
		self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
		self.bn1 = nn.BatchNorm2d(out_channels)
		self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride,
							   padding=1, bias=False)
		self.bn2 = nn.BatchNorm2d(out_channels)
		self.conv3 = nn.Conv2d(out_channels, out_channels * 4, kernel_size=1, bias=False)
		self.bn3 = nn.BatchNorm2d(out_channels * 4)
		self.relu = nn.ReLU(inplace=True)
		self.downsample = downsample
		self.stride = stride

	def forward(self, x):
		residual = x

		out = self.conv1(x)
		out = self.bn1(out)
		out = self.relu(out)

		out = self.conv2(out)
		out = self.bn2(out)
		out = self.relu(out)

		out = self.conv3(out)
		out = self.bn3(out)

		if self.downsample is not None:
			residual = self.downsample(x)

		out += residual
		out = self.relu(out)

		return out


class RGA_Branch(nn.Module):
    def __init__(self,  last_stride=1, block=Bottleneck, layers=[3, 4, 6, 3],
        spa_on=True, cha_on=True, s_ratio=8, c_ratio=8, d_ratio=8, height=256, width=128):
        super(RGA_Branch, self).__init__()

        self.in_channels = 64

            # Networks
        backbone=resnet50(True)
        backbone.layer4[0].downsample[0].stride=1
        backbone.layer4[0].conv2.stride=1
        self.conv1=backbone.conv1
        self.bn1=backbone.bn1
        self.relu=backbone.relu
        self.maxpool=backbone.maxpool
        self.layer1=backbone.layer1
        self.layer2=backbone.layer2
        self.layer3=backbone.layer3
        self.layer4=backbone.layer4
        

        # RGA Modules
        self.rga_att1 = RGA_Module(256, (height//4)*(width//4), use_spatial=spa_on, use_channel=cha_on,
                                cha_ratio=c_ratio, spa_ratio=s_ratio, down_ratio=d_ratio)
        self.rga_att2 = RGA_Module(512, (height//8)*(width//8), use_spatial=spa_on, use_channel=cha_on,
                                cha_ratio=c_ratio, spa_ratio=s_ratio, down_ratio=d_ratio)
        self.rga_att3 = RGA_Module(1024, (height//16)*(width//16), use_spatial=spa_on, use_channel=cha_on,
                                cha_ratio=c_ratio, spa_ratio=s_ratio, down_ratio=d_ratio)
        self.rga_att4 = RGA_Module(2048, (height//16)*(width//16), use_spatial=spa_on, use_channel=cha_on,
                                cha_ratio=c_ratio, spa_ratio=s_ratio, down_ratio=d_ratio)
        
        # Load the pre-trained model weights


    def load_partial_param(self, state_dict, model_index, model_path):
        param_dict = torch.load(model_path)
        for i in state_dict:
            key = 'layer{}.'.format(model_index)+i
            state_dict[i].copy_(param_dict[key])
        del param_dict

    def load_specific_param(self, state_dict, param_name, model_path):
        param_dict = torch.load(model_path)
        for i in state_dict:
            key = param_name + '.' + i
            state_dict[i].copy_(param_dict[key])
        del param_dict

    def forward(self, x):
      
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)

        x = self.rga_att1(x)

        x = self.layer2(x)
        x = self.rga_att2(x)
        
        x = self.layer3(x)
        x = self.rga_att3(x)

        x = self.layer4(x)
    
        x = self.rga_att4(x)

        return x
        

class ResNet50_RGA_Model(nn.Module):
    '''
    Backbone: ResNet-50 + RGA modules.
    '''
    def __init__(self,  num_feat=2048, height=384, width=128, 
        dropout=0, num_classes=0, last_stride=1, branch_name='rgasc', scale=8, d_scale=8):
        super(ResNet50_RGA_Model, self).__init__()
    
        self.num_feat = num_feat
        self.dropout = dropout
        self.num_classes = num_classes
        self.branch_name = branch_name
    
        if 'rgasc' in branch_name:
            spa_on=True 
            cha_on=True
        elif 'rgas' in branch_name:
            spa_on=True
            cha_on=False
        elif 'rgac' in branch_name:
            spa_on=False
            cha_on=True
        else:
            raise NameError
        
        self.backbone = RGA_Branch(last_stride=last_stride, 
                        spa_on=spa_on, cha_on=cha_on, height=height, width=width,
                        s_ratio=scale, c_ratio=scale, d_ratio=d_scale)

        self.feat_bn = nn.BatchNorm1d(self.num_feat)
        self.feat_bn.bias.requires_grad_(False)
        if self.dropout > 0:
            self.drop = nn.Dropout(self.dropout)
        self.cls = nn.Linear(self.num_feat, self.num_classes, bias=False)

        self.feat_bn.apply(weights_init_kaiming)
        self.cls.apply(weights_init_classifier)


    def forward(self, inputs, training=True):
     
        feat_ = self.backbone(inputs)
        feat_ = F.avg_pool2d(feat_, feat_.size()[2:]).view(feat_.size(0), -1)
        feat = self.feat_bn(feat_)
        if self.dropout > 0:
            feat = self.drop(feat)
        if training and self.num_classes is not None:
            cls_feat = self.cls(feat)
        out={}
        out['feat']=feat
        out['logit']=cls_feat
        return out


###<font color=green>***relation network***</font>

In [29]:

def weights_init_kaiming(m):
	classname = m.__class__.__name__
	if classname.find('Linear') != -1:
		nn.init.kaiming_normal_(m.weight, a=0, mode='fan_out')
		nn.init.constant_(m.bias, 0.0)
	elif classname.find('Conv') != -1:
		nn.init.kaiming_normal_(m.weight, a=0, mode='fan_in')
		if m.bias is not None:
			nn.init.constant_(m.bias, 0.0)
	elif classname.find('BatchNorm') != -1:
		if m.affine:
			nn.init.normal_(m.weight, 1.0, 0.02)
			nn.init.constant_(m.bias, 0.0)


def weights_init_fc(m):
	classname = m.__class__.__name__
	if classname.find('Linear') != -1:
		nn.init.normal_(m.weight, std=0.001)
		nn.init.constant_(m.bias, 0.0)
	elif classname.find('BatchNorm') != -1:
		if m.affine:
			nn.init.normal_(m.weight, 1.0, 0.02)
			nn.init.constant_(m.bias, 0.0)
class RelationModel(nn.Module):
  def __init__(self,cfg,num_stripes,num_classes):
    super(RelationModel,self).__init__()
    if cfg['backbone']=='resnet50':
          self.base,in_planes=ResNet50(cfg)
    if cfg['backbone']=='resnet101':
          self.base,in_planes=ResNet101(cfg) 
    if cfg['backbone']=='eff_b3':
          self.base,in_planes=Eff_b3(cfg)  
    if cfg['backbone']=='eff_b5':
          self.base,in_planes=Eff_b5(cfg)
    if cfg['backbone']=='eff_b7':
          self.base,in_planes=Eff_b7(cfg)
    if cfg['backbone']=='eff_b6':
          self.base,in_planes=Eff_b6(cfg)  
    self.num_classes=num_classes
    self.num_stripes=num_stripes
    out_channel=int(in_planes/8)
    self.local_conv_list=nn.ModuleList()
    self.rest_conv_list=nn.ModuleList()
    self.relation_conv_list=nn.ModuleList()
    for i in range(num_stripes):
      self.local_conv_list.append(
          nn.Sequential(
              nn.Conv2d(in_planes,out_channel,1),
              nn.BatchNorm2d(out_channel),
              nn.Relu(inplace=True)
          )
      ) 
    for i in range(num_stripes):
      self.rest_conv_list.append(
          nn.Sequential(
              nn.Conv2d(in_planes,out_channel,1),
              nn.BatchNorm2d(out_channel),
              nn.Relu(inplace=True)
          )
      ) 
    for i in range(num_stripes):
      self.relation_conv_list.append(
          nn.Sequential(
              nn.Conv2d(2*in_planes,out_channel,1),
              nn.BatchNorm2d(out_channel),
              nn.Relu(inplace=True)
          )
      ) 
    
    self.global_max_conv=nn.Sequential(
              nn.Conv2d(in_planes,out_channel,1),
              nn.BatchNorm2d(out_channel),
              nn.Relu(inplace=True)
          )
    self.global_rest_conv=nn.Sequential(
              nn.Conv2d(in_planes,out_channel,1),
              nn.BatchNorm2d(out_channel),
              nn.Relu(inplace=True)
          )
    self.global_pooling_conv=nn.Sequential(
              nn.Conv2d(2*in_planes,out_channel,1),
              nn.BatchNorm2d(out_channel),
              nn.Relu(inplace=True)
          )
    
    

    self.fc_local_list=nn.ModuleList()
    self.fc_rest_list=nn.ModuleList()
    self.fc_relation_list=nn.ModuleList()
    for _ in range(num_stripes):
        fc = nn.Linear(out_channel, num_classes)
        fc.apply(weights_init_fc)
        self.fc_local_list.append(fc)
    for _ in range(num_stripes):
        fc = nn.Linear(out_channel, num_classes)
        fc.apply(weights_init_fc)
        self.fc_rest_list.append(fc)
    for _ in range(num_stripes):
        fc = nn.Linear(out_channel, num_classes)
        fc.apply(weights_init_fc)
        self.fc_relation_list.append(fc)
    
    self.fc_max=nn.Linear(out_channel, num_classes)
    self.fc_max.apply(weights_init_fc)

    self.fc_rest=nn.Linear(out_channel, num_classes)
    self.fc_rest.apply(weights_init_fc)

    self.fc_pooling=nn.Linear(out_channel, num_classes)
    self.fc_pooling.apply(weights_init_fc)

  def forward(self,x):
    feat=self.base(x)
    out=[]
    if self.training:
      stripe_h=int(feat.size(2) / self.num_stripes)
      local_feat_list=[]
      other_feat_list=[]
      final_feat_list=[]
      logit_list=[]
      rest_feat_list=[]
      
      for i in range(self.num_stripes): # 得到6块中每一个的特征
        local_feat = F.max_pool2d(
                  feat[:, :, i * stripe_h: (i + 1) * stripe_h, :], # 每一块是4*w
                  (stripe_h, feat.size(-1))) #pool成1*1的
              #print(local_6_feat.shape) #8 2048 1 1
              
        local_feat_list.append(local_feat)

      for i in range(self.num_stripes): #对于每块特征，除去自己之后其他的特征组合在一起
              
              rest_feat_list.append((local_feat_list[(i+1)%self.num_stripes]#论文公式1处的ri 
                                    + local_feat_list[(i+2)%self.num_stripes]
                                    + local_feat_list[(i+3)%self.num_stripes] 
                                    + local_feat_list[(i+4)%self.num_stripes]
                                    + local_feat_list[(i+5)%self.num_stripes])/5)
              
      global_max_feat = F.max_pool2d(feat, (feat.size(2), feat.size(3)))
      global_rest_feat = (local_feat_list[0] + local_feat_list[1] + local_feat_list[2]#局部与全局的差异 
                                + local_feat_list[3] + local_feat_list[4] + local_feat_list[5] - global_max_feat)/5
      global_max_feat = self.global_max_conv(global_max_feat)
      global_rest_feat=self.global_rest_conv(global_rest_feat)

      global_pooling_feat=self.global_pooling_conv(torch.cat((global_max_feat,global_rest_feat),1))
      
      

      global_feat=(global_pooling_feat+global_max_feat)/2
      final_feat_list.append(global_feat)
      other_feat_list.append(global_max_feat)
      other_feat_list.append(global_rest_feat)

      global_max_logit=self.fc_max(global_max_feat)
      global_rest_logit=self.fc_rest(global_rest_feat)
      global_pooling_logit=self.fc_pooling(global_feat)
      logit_list.append(global_max_logit)
      logit_list.append(global_rest_logit)
      logit_list.append(global_pooling_logit)
      
      for i in range(self.num_stripes):
        local_feat=self.local_conv_list[i](local_feat_list[i]).squeeze(3).squeeze(2)
        local_rest_feat=self.rest_conv_list[i](rest_feat_list[i]).squeeze(3).squeeze(2)
        relation_feat=self.relation_conv_list[i](torch.cat((local_feat,local_rest_feat),1).unsqueeze(2).unsqueeze(3))
        relation_feat=(relation_feat+local_feat.unsqueeze(2).unsqueeze(3)).squeeze(3).squeeze(2)

        final_feat_list.append(relation_feat)
        other_feat_list.append(local_feat)
        other_feat_list.append(local_rest_feat)

        local_logits=self.fc_local_list[i](local_feat)
        rest_logit=self.fc_rest_list[i](local_rest_feat)
        relation_logit=self.fc_relation_list[i](relation_feat)

        logit_list.append(local_logits)
        logit_list.append(rest_logit)
        logit_list.append(relation_logit)
    
    
      final_feat_list.extend(other_feat_list)
      out['feat']=final_feat_list
      out['logit']=logit_list
    else:
      output_feat=0
      for i in range(self.num_stripes): # 得到6块中每一个的特征
        local_feat = F.max_pool2d(
                  feat[:, :, i * stripe_h: (i + 1) * stripe_h, :], # 每一块是4*w
                  (stripe_h, feat.size(-1))) #pool成1*1的
              #print(local_6_feat.shape) #8 2048 1 1
              
        local_feat_list.append(local_feat)

      for i in range(self.num_stripes): #对于每块特征，除去自己之后其他的特征组合在一起
              
              rest_feat_list.append((local_feat_list[(i+1)%self.num_stripes]#论文公式1处的ri 
                                    + local_feat_list[(i+2)%self.num_stripes]
                                    + local_feat_list[(i+3)%self.num_stripes] 
                                    + local_feat_list[(i+4)%self.num_stripes]
                                    + local_feat_list[(i+5)%self.num_stripes])/5)
              
      global_max_feat = F.max_pool2d(feat, (feat.size(2), feat.size(3)))
      global_rest_feat = (local_feat_list[0] + local_feat_list[1] + local_feat_list[2]#局部与全局的差异 
                                + local_feat_list[3] + local_feat_list[4] + local_feat_list[5] - global_max_feat)/5
      global_max_feat = self.global_max_conv(global_max_feat)
      global_rest_feat=self.global_rest_conv(global_rest_feat)

      global_pooling_feat=self.global_pooling_conv(torch.cat(global_max_feat,global_rest_feat,1))
      
      

      global_feat=(global_pooling_feat+global_max_feat)/2
      output_feat=global_feat
      for i in range(self.num_stripes):
        local_feat=self.local_conv_list[i](local_feat_list[i]).squeeze(3).squeeze(2)
        local_rest_feat=self.rest_conv_list[i](rest_feat_list[i]).squeeze(3).squeeze(2)
        relation_feat=self.relation_conv_list[i](torch.cat(local_feat,local_rest_feat,1).unsqueeze(2).unsqueeze(3))
        relation_feat=((relation_feat+local_feat.unsqueeze(2).unsqueeze(3)).squeeze(3).squeeze(2))/2
        output_feat=torch.cat((output_fet,relation_feat),1)

      out['feat']=[output_feat]
    return out
    



    

    
    




#<font color=red>***metric***</font>

##<font color=blue>***图像***</font>

###<font color=green>***普通map***

In [30]:

'''reid计算map函数'''
def MARKET_EVAL_FUNC(dismat,q_pids,g_pids,q_camids,g_camids,max_rank=50):
    '''
    :param dismat: 每个query与其对应查询结果的距离
    :param q_pids: 每个query行人的id
    :param g_pids: 每个查询结果的行人的id
    :param q_camids: 摄像头id，对于同一个query，如果查询结果与query都来自一个摄像头，则丢弃
    :param g_camids: 同上
    :param max_rank:
    :return:
    '''
    num_q,num_g=dismat.shape
    if num_g<max_rank:
        max_rank=num_g

    ##根据相似度进行排序
    indices=np.argsort(dismat,axis=1)
    
    
    ##在排序结果找到同id图片，用于计算cmc
    matchs=(g_pids[indices]==q_pids[:,np.newaxis]).astype(np.int32)
    


    all_cmc=[]
    all_AP=[]
    num_valid_q=0

    for q_idx in range(num_q):
        q_pid=q_pids[q_idx]
        q_camid=q_camids[q_idx]

        '''当前query对应查询结果的排序'''
        order=indices[q_idx]

        '''若查询结果与query同摄像机且同id，删除'''
        remove=(g_pids[order]==q_pid)&(g_camids[order]==q_camid)
        keep=np.invert(remove)

        '''计算CMC'''

        '''首先先筛选'''
        orig_cmc=matchs[q_idx][keep]

        '''全是false'''
        if not np.any(orig_cmc):
            continue

        

        '''累加和，因为orig_cmc里面是true，false，true为1，得到cmc列表'''
        cmc=orig_cmc.cumsum()
        cmc[cmc>1]=1
        all_cmc.append(cmc[:max_rank])
        num_valid_q+=1

        '''计算map'''

        '''预测正确的数量'''
        num_rel=orig_cmc.sum()

        '''累加和，用于计算'''
        tmp_cmc=orig_cmc.cumsum()

        '''计算准确率，前k个查询结果的准确率'''
        tmp_cmc=[x/(i+1) for i,x in enumerate(tmp_cmc)]

        '''计算召回率，召回率每次变化都是当查询结果id等于query时，故乘orig_cmc'''
        '''最终的结果类似 0,0,1,0,0,2,0,0,3这种，每个有值的都是查询正确的'''
        '''然后后面除以查询结果中同id数量，也就是上面num_rel就是找召回率'''
        '''但是这里其实已经提前乘上准确率了，最终结果类似于 0,0,1/3,0,0,2/6,0,0,3/9'''
        tmp_cmc=np.asarray(tmp_cmc)*orig_cmc

        '''以上为例，除以同id数量后，假设为五个'''
        '''0，0，1/3 /5，0，0，2/6 /5，0，0，3/9 /5'''
        '''等价于 0,0, 1/3(准确率)*1/5(召回率) ....'''
        
        if num_rel==0:
          AP=0
          print('AP=0')
        else:  
          AP = tmp_cmc.sum() / num_rel
          
        all_AP.append(AP)

    all_cmc = np.asarray(all_cmc).astype(np.float32)
    all_cmc = all_cmc.sum(0) / num_valid_q
    if len(all_AP)==0:
      print('all_ap=0')
      return all_cmc,0
    mAP = np.mean(all_AP)

    return all_cmc, mAP


class MARKET_MAP():
    def __init__(self,num_query,max_rank=50,feat_norm=True,one_day=True,all=False,date_length=11):
        '''

        :param num_query: 查询数量
        :param max_rank: 每个query限定的最大查询结果数量
        :param feat_norm:是否对特征进行l2norm
        '''
        self.num_query=num_query
        self.max_rank=max_rank
        self.feat_norm=feat_norm
        self.one_day=one_day
        self.all=all
        self.date_length=date_length
        self.reset()

    def reset(self):
        '''
        feats:模型返回的特征
        pids: 图片中行人的id
        camids:图片摄像头的id
        :return:
        '''
        self.feats=[]
        self.pids=[]
        self.camids=[]
        self.dates=[]

    def update(self,output):
        feat,pid,camid,date=output
    
        self.feats.append(feat)
        self.pids.append(pid)
        self.camids.append(camid)
        self.dates.append(date)

    def compute(self):
        feats=torch.stack(self.feats,dim=0)
          
        if self.feat_norm:
            feats=torch.nn.functional.normalize(feats,p=2)

        
        '''用于查询的图片的特征及其id和摄像头id'''
        
        qf=feats[:self.num_query]
        q_pids=np.asarray(self.pids[:self.num_query])
        q_camids = np.asarray(self.camids[:self.num_query])
        q_dates=np.asarray(self.dates[:self.num_query])  

        '''每个query查询结果的特征及其id和摄像头id'''
        gf = feats[self.num_query:]
        g_pids = np.asarray(self.pids[self.num_query:])
        g_camids = np.asarray(self.camids[self.num_query:])
        g_dates=np.asarray(self.dates[self.num_query:])
        if self.all:
          m,n=qf.shape[0],gf.shape[0]
      
          '''计算qf每个query和对应查询结果的距离'''
          distmat = torch.pow(qf, 2).sum(dim=1, keepdim=True).expand(m, n) + \
                    torch.pow(gf, 2).sum(dim=1, keepdim=True).expand(n, m).t()
        
          distmat.addmm_(1, -2, qf, gf.t())
          distmat = distmat.cpu().numpy()
        
          cmc,mAP=MARKET_EVAL_FUNC(distmat,q_pids,g_pids,q_camids,g_camids)
          return mAP
        else:
          if self.one_day:
            
            mean_map=0
            count=0
            for date in range(self.date_length):
              date=date+1
              q_index=np.where(q_dates==date)
              
              date_qf=qf[q_index]
              date_q_pids=q_pids[q_index]
              date_q_camids=q_camids[q_index]

              g_index=np.where(g_dates==date)
              date_gf=gf[g_index]
              date_g_pids=g_pids[g_index]
              date_g_camids=g_camids[g_index]
              m,n=date_qf.shape[0],date_gf.shape[0]

              if m==0:
                continue
              distmat = torch.pow(date_qf, 2).sum(dim=1, keepdim=True).expand(m, n) + \
                    torch.pow(date_gf, 2).sum(dim=1, keepdim=True).expand(n, m).t()
              
              distmat.addmm_(1, -2, date_qf, date_gf.t())
              distmat = distmat.cpu().numpy()
              
              cmc,mAP=MARKET_EVAL_FUNC(distmat,date_q_pids,date_g_pids,date_q_camids,date_g_camids)
              mean_map+=mAP
              count+=1
            
            if count==0:
              return 0
            return mean_map/count
          else:
           
            mean_map=0
            count=0
            for date in range(self.date_length):
              date=date+1
              q_index=np.where(q_dates==date)
              
              date_qf=qf[q_index]
              date_q_pids=q_pids[q_index]
              date_q_camids=q_camids[q_index]

              g_index=np.where(g_dates!=date)
              date_gf=gf[g_index]
              date_g_pids=g_pids[g_index]
              date_g_camids=g_camids[g_index]
              m,n=date_qf.shape[0],date_gf.shape[0]

              if m==0 or n==0:
                continue
              distmat = torch.pow(date_qf, 2).sum(dim=1, keepdim=True).expand(m, n) + \
                    torch.pow(date_gf, 2).sum(dim=1, keepdim=True).expand(n, m).t()
              
              distmat.addmm_(1, -2, date_qf, date_gf.t())
              distmat = distmat.cpu().numpy()
              
              cmc,mAP=MARKET_EVAL_FUNC(distmat,date_q_pids,date_g_pids,date_q_camids,date_g_camids)
              mean_map+=mAP
              count+=1
            if count==0:
              return 0
            return mean_map/count




###<font color=green>***rerank map***

In [31]:
def RE_RANKING(porbFea,galFea,k1,k2,lambda_value,local_distmat=None,only_local=False):
    '''
    :param porbFea:query特征
    :param galFea: 查询结果特征
    :param k1: 一开始k近邻的k
    :param k2:
    :param lambda_value:
    :param locl_distmat:
    :param only_loca:
    :return:
    '''
    query_num=porbFea.shape[0]
    all_num=query_num+galFea.shape[0]
    if only_local:
        original_dist=local_distmat
    else:
        '''把query加进去，然后算每个图片之间的距离，用于后面reranking'''
        feat=torch.cat([porbFea,galFea])
        distmat = torch.pow(feat, 2).sum(dim=1, keepdim=True).expand(all_num, all_num) + \
                  torch.pow(feat, 2).sum(dim=1, keepdim=True).expand(all_num, all_num).t()
        distmat.addmm_(1, -2, feat, feat.t())
        original_dist = distmat.cpu().numpy()
        del feat
        if not local_distmat is None:
            original_dist = original_dist + local_distmat
    gallery_num=original_dist.shape[0]

    '''进行归一化，然后进行转置，得到归一化的距离矩阵'''
    original_dist = np.transpose(original_dist / np.max(original_dist, axis=0))

    V=np.zeros_like(original_dist).astype(np.float16)

    '''初始rank列表就是直接根据距离最小值'''
    initial_rank=np.argsort(original_dist).astype(np.int32)

    '''进行重排序'''
    '''最核心的思想就是对于query，前k个距离最近的查询结果，每一个查询结果在所有图片中的k近邻
    应该包含query，根据这个进行重排，使得rerank后尽量满足这个条件'''
    for i in range(all_num):
        '''选择当前图片的k近邻'''
        forward_k_neigh_index=initial_rank[i,:k1+1]

        '''选择k近邻的k近邻'''
        backward_k_neigh_index=initial_rank[forward_k_neigh_index,:k1+1]

        '''满足上面的条件，k近邻的k近邻包含当前图片'''
        fi=np.where(backward_k_neigh_index==i)[0]

        '''那么满足上面条件的k近邻作为rerank的候选集'''
        k_reciprocal_index=forward_k_neigh_index[fi]
        k_reciprocal_expansion_index = k_reciprocal_index

        '''如果满足上面的条件，通常就是正样本无误了，这也就是rerank的目的'''
        '''但是问题在于如果因为一些干扰，导致正样本在k1近邻之外，那rerank就没用'''
        '''所以提出一个方法进行候选集的扩充，希望能够保证正样本进入候选集'''

        '''基本的思想就是对于当前图片，我们找到了符合条件的原候选集'''
        '''那么对于候选集每一个元素，找到以一半k1为长度的满足上面条件的候选集'''
        '''如果这个候选集与原候选集的交集超过这个候选集的2/3长度，那么把这个候选集加入原候选集'''
        for j in range(len(k_reciprocal_index)):
            '''原候选集每个元素'''
            candidate=k_reciprocal_index[j]
            '''找到当前元素满足条件的候选集'''
            candidate_forward_k_neigh_index = initial_rank[candidate, :int(np.around(k1 / 2)) + 1]
            candidate_backward_k_neigh_index = initial_rank[candidate_forward_k_neigh_index,
                                               :int(np.around(k1 / 2)) + 1]
            fi_candidate = np.where(candidate_backward_k_neigh_index == candidate)[0]
            candidate_k_reciprocal_index = candidate_forward_k_neigh_index[fi_candidate]

            '''如果交集长度超过这个候选集的2/3长度'''
            if len(np.intersect1d(candidate_k_reciprocal_index,k_reciprocal_index))>(2/3)*len(candidate_k_reciprocal_index):
                k_reciprocal_expansion_index=np.append(k_reciprocal_expansion_index,candidate_k_reciprocal_index)
        k_reciprocal_expansion_index = np.unique(k_reciprocal_expansion_index)

        '''获取当前图片及其候选集的距离，进行exp'''
        weight = np.exp(-original_dist[i, k_reciprocal_expansion_index])

        '''找到每个图片与其候选集的一个权值表示，用于后面计算jaccard距离'''
        V[i, k_reciprocal_expansion_index] = weight / np.sum(weight)
    original_dist=original_dist[:query_num,]

    '''这部分处理时针对线上reid系统，因为gallery通常是一直更新的，由于计算V的过程比较麻烦'''
    '''我们只能隔一段时间计算，那么对于新加入的图片，其在V中没有值，所以采用这个近似方法来计算'''
    if k2!=1:
        V_qe=np.zeros_like(V,dtype=np.float16)
        for i in range(all_num):
            '''采用查询样本的特征均值代替上面的reciprocal编码，只是近似处理'''
            V_qe[i,:]=np.mean(V[initial_rank[i,:k2],:],axis=0)
        V=V_qe
        del V_qe
    del initial_rank
    invIndex=[]
    for i in range(gallery_num):
        '''找到候选集包含当前图片的所有query'''
        invIndex.append(np.where(V[:,i]!=0)[0])

    '''计算jaccard距离'''
    jaccard_dist=np.zeros_like(original_dist,dtype=np.float16)

    '''jaccard距离的计算，对于query q和其中一个查询样本gi的距离，我们用1-其候选集的交并比来代表，如果交并比越大，代表两者'''
    '''候选集很接近，说明两者很接近，则其距离就很小'''

    '''为了方便表示交集和并集，我们建立Vq，其中V(q,gi)代表gi是否在q候选集中，如果在为1，否则为0，那么这个Vq就是针对所有图片，类似候选集的one-hot'''
    '''Vgi同理，但是one-hot是hard编码，为了soft编码，我们使用上面np.exp(-距离）的计算'''

    '''然后交集怎么表示呢？ sum(min(vq,vgi))，就是每个位置取Vq和Vgi的最小值,然后求和，为什么呢？我们以one-hot去想，交集意味着两者一样，那么对于相同位置，只有Vq和Vgi在这个位置
    都为1，那么这个位置才为1，否则为0，所以求和的时候就是求1的个数，相当于求对应位置是相同的个数，这就是交集的定义'''

    '''并集是sum(max(Vq,Vgi)),意义同上。'''

    for i in range(query_num):
        temp_min=np.zeros(shape=[1,gallery_num],dtype=np.float16)
        '''找到当前图片的候选集'''
        indNonZero=np.where(V[i,:]!=0)[0]

        '''候选集的候选集'''
        indImages=[invIndex[ind] for ind in indNonZero]
        '''交集的计算'''
        '''不断求和，看公式'''
        for j in range(len(indNonZero)):
               temp_min[0, indImages[j]] = temp_min[0, indImages[j]] + np.minimum(V[i, indNonZero[j]],
                                                                                       V[indImages[j], indNonZero[j]])
        '''2-temp_min是并集的快速计算'''
        jaccard_dist[i] = 1 - temp_min / (2 - temp_min)


    '''作为最终的距离矩阵'''
    final_dist = jaccard_dist * (1 - lambda_value) + original_dist * lambda_value
    del original_dist
    del V
    del jaccard_dist
    final_dist = final_dist[:query_num, query_num:]
    return final_dist



'''使用reranking技术后的准确度'''
class MARKET_mAP_reranking():
    def __init__(self, num_query, max_rank=50, feat_norm=True):
        self.num_query = num_query
        self.max_rank = max_rank
        self.feat_norm = feat_norm
        self.reset()

    def reset(self):
        self.feats = []
        self.pids = []
        self.camids = []

    def update(self, output):
        feat, pid, camid = output
        self.feats.append(feat)
        self.pids.append(pid)
        self.camids.append(camid)

    def compute(self):
        feats = torch.stack(self.feats, dim=0)
        if self.feat_norm :
           
            feats = torch.nn.functional.normalize(feats, p=2)

        # query
        qf = feats[:self.num_query]
        q_pids = np.asarray(self.pids[:self.num_query])
        q_camids = np.asarray(self.camids[:self.num_query])
        # gallery
        gf = feats[self.num_query:]
        g_pids = np.asarray(self.pids[self.num_query:])
        g_camids = np.asarray(self.camids[self.num_query:])
        # m, n = qf.shape[0], gf.shape[0]
        # distmat = torch.pow(qf, 2).sum(dim=1, keepdim=True).expand(m, n) + \
        #           torch.pow(gf, 2).sum(dim=1, keepdim=True).expand(n, m).t()
        # distmat.addmm_(1, -2, qf, gf.t())
        # distmat = distmat.cpu().numpy()
        print("Enter reranking")
        distmat = RE_RANKING(qf, gf, k1=20, k2=6, lambda_value=0.3)
        cmc, mAP = MARKET_EVAL_FUNC(distmat, q_pids, g_pids, q_camids, g_camids)

        return cmc, mAP

# <font color=red>***Loss***


##<font color=blue>***分类损失***</font>

In [32]:
class CrossEntropyLabelSmoothLoss(nn.Module):
	"""Cross entropy loss with label smoothing regularizer.
	Reference:
	Szegedy et al. Rethinking the Inception Architecture for Computer Vision. CVPR 2016.
	Equation: y = (1 - epsilon) * y + epsilon / K.
	Args:
		num_classes (int): number of classes.
		epsilon (float): weight.
	"""
	def __init__(self, num_classes, epsilon=0.1, use_gpu=True):
		super(CrossEntropyLabelSmoothLoss, self).__init__()
		self.num_classes = num_classes
		self.epsilon = epsilon
		self.use_gpu = use_gpu
		self.logsoftmax = nn.LogSoftmax(dim=1)

	def forward(self, inputs, targets):
		"""
		Args:
			inputs: prediction matrix (before softmax) with shape (batch_size, num_classes)
			targets: ground truth labels with shape (num_classes)
		"""
 
		log_probs = self.logsoftmax(inputs['logit'])
		targets = torch.zeros(log_probs.size()).scatter_(1, targets.unsqueeze(1).cpu(), 1)
		if self.use_gpu: targets = targets.cuda()
		targets = (1 - self.epsilon) * targets + self.epsilon / self.num_classes
		loss = (- targets * log_probs).mean(0).sum()
		return loss

## <font color=blue>***度量学习***

###<font color=green>***triplet***

In [33]:
  
import torch
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F

import numpy as np


def normalize(x, axis=-1):
	"""Normalizing to unit length along the specified dimension.
	Args:
	  x: pytorch Variable
	Returns:
	  x: pytorch Variable, same shape as input
	"""
	x = 1. * x / (torch.norm(x, 2, axis, keepdim=True).expand_as(x) + 1e-12)
	return x

def euclidean_dist(x, y):
	"""
	Args:
	  x: pytorch Variable, with shape [m, d]
	  y: pytorch Variable, with shape [n, d]
	Returns:
	  dist: pytorch Variable, with shape [m, n]
	"""
	m, n = x.size(0), y.size(0)
	xx = torch.pow(x.float(), 2).sum(1, keepdim=True).expand(m, n)
	yy = torch.pow(y.float(), 2).sum(1, keepdim=True).expand(n, m).t()
	dist = xx + yy
	dist.addmm_(1, -2, x.float(), y.t().float())
	dist = dist.clamp(min=1e-12).sqrt()  # for numerical stability
	return dist

def cosine_dist(x, y):
	"""
	Args:
	  x: pytorch Variable, with shape [m, d]
	  y: pytorch Variable, with shape [n, d]
	"""
	x_normed = F.normalize(x, p=2, dim=1)
	y_normed = F.normalize(y, p=2, dim=1)
	return 1 - torch.mm(x_normed, y_normed.t())

def cosine_similarity(x, y):
	"""
	Args:
	  x: pytorch Variable, with shape [m, d]
	  y: pytorch Variable, with shape [n, d]
	"""
	x_normed = F.normalize(x, p=2, dim=1)
	y_normed = F.normalize(y, p=2, dim=1)
	return torch.mm(x_normed, y_normed.t())


def hard_example_mining(dist_mat, labels, return_inds=False):
	"""For each anchor, find the hardest positive and negative sample.
	Args:
	  dist_mat: pytorch Variable, pair wise distance between samples, shape [N, N]
	  labels: pytorch LongTensor, with shape [N]
	  return_inds: whether to return the indices. Save time if `False`(?)
	Returns:
	  dist_ap: pytorch Variable, distance(anchor, positive); shape [N]
	  dist_an: pytorch Variable, distance(anchor, negative); shape [N]
	  p_inds: pytorch LongTensor, with shape [N];
		indices of selected hard positive samples; 0 <= p_inds[i] <= N - 1
	  n_inds: pytorch LongTensor, with shape [N];
		indices of selected hard negative samples; 0 <= n_inds[i] <= N - 1
	NOTE: Only consider the case in which all labels have same num of samples,
	  thus we can cope with all anchors in parallel.
	"""
	assert len(dist_mat.size()) == 2
	assert dist_mat.size(0) == dist_mat.size(1)
	N = dist_mat.size(0)

	is_pos = labels.expand(N, N).eq(labels.expand(N, N).t())
	is_neg = labels.expand(N, N).ne(labels.expand(N, N).t())

	dist_ap, relative_p_inds = torch.max(
		dist_mat[is_pos].contiguous().view(N, -1), 1, keepdim=True)
	dist_an, relative_n_inds = torch.min(
		dist_mat[is_neg].contiguous().view(N, -1), 1, keepdim=True)
	
	dist_ap = dist_ap.squeeze(1)
	dist_an = dist_an.squeeze(1)

	if return_inds:
		ind = (labels.new().resize_as_(labels)
			   .copy_(torch.arange(0, N).long())
			   .unsqueeze(0).expand(N, N))
		p_inds = torch.gather(
			ind[is_pos].contiguous().view(N, -1), 1, relative_p_inds.data)
		n_inds = torch.gather(
			ind[is_neg].contiguous().view(N, -1), 1, relative_n_inds.data)
		p_inds = p_inds.squeeze(1)
		n_inds = n_inds.squeeze(1)
		return dist_ap, dist_an, p_inds, n_inds

	return dist_ap, dist_an



class TripletHardLoss(object):
	"""Modified from Tong Xiao's open-reid (https://github.com/Cysu/open-reid).
	Related Triplet Loss theory can be found in paper 'In Defense of the Triplet
	Loss for Person Re-Identification'."""
	def __init__(self, margin=None, metric="euclidean"):
		self.margin = margin
		self.metric = metric
		if margin is not None:
			self.ranking_loss = nn.MarginRankingLoss(margin=margin)
		else:
			self.ranking_loss = nn.SoftMarginLoss()

	def __call__(self, outs, labels, normalize_feature=True):
   
		if normalize_feature:
			global_feat = normalize(outs['feat'], axis=-1)
     
		if self.metric == "euclidean":
			dist_mat = euclidean_dist(outs['feat'], global_feat)
		elif self.metric == "cosine":
			dist_mat = cosine_dist(outs['feat'], global_feat)
		else:
			raise NameError

		dist_ap, dist_an = hard_example_mining(
			dist_mat, labels)
		y = dist_an.new().resize_as_(dist_an).fill_(1)
		
		if self.margin is not None:
			loss = self.ranking_loss(dist_an, dist_ap, y)
		else:
			loss = self.ranking_loss(dist_an - dist_ap, y)
		prec = (dist_an.data > dist_ap.data).sum() * 1. / y.size(0)
		return loss

### <font color=green>***center loss***

In [34]:

class CenterLoss(nn.Module):
    """Center loss.
    Reference:
    Wen et al. A Discriminative Feature Learning Approach for Deep Face Recognition. ECCV 2016.
    Args:
        num_classes (int): number of classes.
        feat_dim (int): feature dimension.
    """

    def __init__(self, num_classes=751, feat_dim=2048, use_gpu=True):
        super(CenterLoss, self).__init__()
        self.num_classes = num_classes
        self.feat_dim = feat_dim
        self.use_gpu = use_gpu

        if self.use_gpu:
            self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim).cuda())
        else:
            self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim))

    def forward(self, x, labels):
        """
        Args:
            x: feature matrix with shape (batch_size, feat_dim).
            labels: ground truth labels with shape (num_classes).
        """
        x=x['feat']
        assert x.size(0) == labels.size(0), "features.size(0) is not equal to labels.size(0)"

        batch_size = x.size(0)
        distmat = torch.pow(x, 2).sum(dim=1, keepdim=True).expand(batch_size, self.num_classes) + \
                  torch.pow(self.centers, 2).sum(dim=1, keepdim=True).expand(self.num_classes, batch_size).t()
        distmat.addmm_(1, -2, x, self.centers.t())

        classes = torch.arange(self.num_classes).long()
        if self.use_gpu: classes = classes.cuda()
        labels = labels.unsqueeze(1).expand(batch_size, self.num_classes)
        mask = labels.eq(classes.expand(batch_size, self.num_classes))

        dist = distmat * mask.float()
        loss = dist.clamp(min=1e-12, max=1e+12).sum() / batch_size
        #dist = []
        #for i in range(batch_size):
        #    value = distmat[i][mask[i]]
        #    value = value.clamp(min=1e-12, max=1e+12)  # for numerical stability
        #    dist.append(value)
        #dist = torch.cat(dist)
        #loss = dist.mean()
        return loss

###<font color=green>***ms_loss***

In [35]:
class MultiSimilarityLoss(nn.Module):
    def __init__(self):
        super(MultiSimilarityLoss, self).__init__()
        self.thresh = 0.5

        '''自我相似度的margin'''
        self.margin = 0.1

        '''正例对的系数，也就是alpha'''
        self.scale_pos = 2

        '''负例对的系数，也就是beta'''
        self.scale_neg = 50

        self.count = 0

    def forward(self, feats, labels):
        assert feats.size(0) == labels.size(0), \
            f"feats.size(0): {feats.size(0)} is not equal to labels.size(0): {labels.size(0)}"
        feats=feats['feat']
        feats=normalize(feats,axis=-1)
        batch_size = feats.size(0)

        '''相似性矩阵，Sij=第i行元素与第j行元素逐元素相乘再相加'''
        sim_mat = torch.matmul(feats, torch.t(feats))


        epsilon = 1e-5
        loss = list()

        for i in range(batch_size):

            '''获取正例对的相似矩阵'''
            pos_pair_ = sim_mat[i][labels == labels[i]]

            '''去除掉自己与自己的正例对'''
            pos_pair_ = pos_pair_[pos_pair_ < (1 - epsilon)]
            if len(pos_pair_) == 0:
                print('MS False  count:{}'.format(self.count))
                self.count += 1
                continue

            '''获取反例对'''
            neg_pair_ = sim_mat[i][labels != labels[i]]

            '''根据positive relative similiarty来进行pair mining,具体看论文笔记'''
            neg_pair = neg_pair_[neg_pair_ + self.margin > min(pos_pair_)]
            pos_pair = pos_pair_[pos_pair_ - self.margin < max(neg_pair_)]

            if len(neg_pair) < 1 or len(pos_pair) < 1:
                continue

            # weighting step
            '''然后就是根据最终公式用代码实现'''
            pos_loss = 1.0 / self.scale_pos * torch.log(
                1 + torch.sum(torch.exp(-self.scale_pos * (pos_pair - self.thresh))))
            neg_loss = 1.0 / self.scale_neg * torch.log(
                1 + torch.sum(torch.exp(self.scale_neg * (neg_pair - self.thresh))))
            loss.append(pos_loss + neg_loss)

        if len(loss) == 0:
            return torch.zeros([], requires_grad=True)

        loss = sum(loss) / batch_size
        return loss



### circle loss


In [36]:
from typing import Tuple
from torch import Tensor
def convert_label_to_similarity(normed_feature: Tensor, label: Tensor) -> Tuple[Tensor, Tensor]:
    similarity_matrix = normed_feature @ normed_feature.transpose(1, 0)
    label_matrix = label.unsqueeze(1) == label.unsqueeze(0)

    positive_matrix = label_matrix.triu(diagonal=1)
    negative_matrix = label_matrix.logical_not().triu(diagonal=1)

    similarity_matrix = similarity_matrix.view(-1)
    positive_matrix = positive_matrix.view(-1)
    negative_matrix = negative_matrix.view(-1)
    return similarity_matrix[positive_matrix], similarity_matrix[negative_matrix]


class CircleLoss(nn.Module):
    def __init__(self, m: float, gamma: float) -> None:
        super(CircleLoss, self).__init__()
        self.m = m
        self.gamma = gamma
        self.soft_plus = nn.Softplus()

    def forward(feat,label) -> Tensor:
        feat=feat['feat']
        feat=normalize(feat,axis=-1)
        sp, sn = convert_label_to_similarity(feat, lbl)
        ap = torch.clamp_min(- sp.detach() + 1 + self.m, min=0.)
        an = torch.clamp_min(sn.detach() + self.m, min=0.)

        delta_p = 1 - self.m
        delta_n = self.m

        logit_p = - ap * (sp - delta_p) * self.gamma
        logit_n = an * (sn - delta_n) * self.gamma

        loss = self.soft_plus(torch.logsumexp(logit_n, dim=0) + torch.logsumexp(logit_p, dim=0))

        return loss




##训练loss

In [37]:
def make_loss(cfg,num_classes):
  loss=cfg['loss']
  criterion=[]
  for name,weight in loss:
    if name=='cross':
      criterion.append((CrossEntropyLabelSmoothLoss(num_classes),weight))
    elif name=='tri':
      criterion.append((TripletHardLoss(cfg['margin']),weight))
    elif name=='ms':
      criterion.append((MultiSimilarityLoss(),weight))
    elif name=='circle':
      criterion.append((CircleLoss(m=0.25, gamma=80),weight))
  return criterion


# <font color=red>***lr_scheduler***</font>

## <font color=blue>***warm up***

In [38]:
from bisect import bisect_right
import torch




class WarmupMultiStepLR(torch.optim.lr_scheduler._LRScheduler):
    def __init__(
        self,
        optimizer,
        milestones,
        gamma=0.1,
        warmup_factor=1.0 / 3,
        warmup_iters=500,
        warmup_method="linear",
        last_epoch=-1,
    ):
        if not list(milestones) == sorted(milestones):
            raise ValueError(
                "Milestones should be a list of" " increasing integers. Got {}",
                milestones,
            )

        if warmup_method not in ("constant", "linear"):
            raise ValueError(
                "Only 'constant' or 'linear' warmup_method accepted"
                "got {}".format(warmup_method)
            )
        self.milestones = milestones
        self.gamma = gamma
        self.warmup_factor = warmup_factor
        self.warmup_iters = warmup_iters
        self.warmup_method = warmup_method
        super(WarmupMultiStepLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        warmup_factor = 1
        if self.last_epoch < self.warmup_iters:
            if self.warmup_method == "constant":
                warmup_factor = self.warmup_factor
            elif self.warmup_method == "linear":
                alpha = self.last_epoch / self.warmup_iters
                warmup_factor = self.warmup_factor * (1 - alpha) + alpha
        return [
            base_lr
            * warmup_factor
            * self.gamma ** bisect_right(self.milestones, self.last_epoch)
            for base_lr in self.base_lrs
        ]            
        

##<font color=blue>***Flat***</font>

In [39]:
class Flat(torch.optim.lr_scheduler._LRScheduler):
  def __init__(self,optimizer,cfg,anneal_start=0.65,last_epoch=-1):
    self.epochs=cfg['epochs']
    self.start=anneal_start
    super(Flat,self).__init__(optimizer,last_epoch)
  
  def get_lr(self):
    if self.last_epoch<self.epochs*self.start:
      return [base_lr for base_lr in self.base_lrs]
    else:
      return [
        base_lr*(1+math.cos(math.pi*self.last_epoch/5))/2  
        for base_lr in self.base_lrs    
      ]
     



##<font color=blue>***不变化的scheduler***

In [40]:
class Unchange_sch:
  def __init__(self):
    pass
  
  def step(self):
    pass

#<font color=red>***自定义optimizer***</font>

#<font color=red>***自定义eva***

##<font color=blue>***普通***

In [41]:
def do_eva1(cfg,model,val_loader,num_query):
  model.eval()
  device=torch.device('cuda:0')
  
  metric0=MARKET_MAP(num_query,all=False,one_day=True)
  metric1=MARKET_MAP(num_query,all=False,one_day=False)

  
  with torch.no_grad():
    for imgs,pids,cams,dates in val_loader:
      imgs=imgs.to(device)
      outs=model(imgs)
      
      for out in zip(outs['feat'],pids,cams,dates):
          metric0.update(out)
          metric1.update(out)
         
      

  map0=metric0.compute()
  map1=metric1.compute()
  
  
  return map0,map1




#<font color=red>***训练辅助函数***</font>

##<font color=blue>***checkpoint***</font>

In [42]:
class Checkpoint:
  def __init__(self,ckpt):
    self.ckpt=ckpt
    self.init=-1000
  
  def __call__(self,metric,model,optimizer):
    if metric>self.init:
      self.init=metric
      torch.save({'model':model.state_dict(),'optimizer':optimizer.state_dict()},self.ckpt)
    


##<font color=blue>***EMA***

In [43]:
from copy import deepcopy
from collections import OrderedDict
from sys import stderr

# for type hint
from torch import Tensor


class EMA(nn.Module):
    def __init__(self, model: nn.Module, decay: float):
        super().__init__()
        self.decay = decay

        self.model = model
        self.shadow = deepcopy(self.model)

        for param in self.shadow.parameters():
            param.detach_()

    @torch.no_grad()
    def update(self):
        if not self.training:
            print("EMA update should only be called during training", file=stderr, flush=True)
            return

        model_params = OrderedDict(self.model.named_parameters())
        shadow_params = OrderedDict(self.shadow.named_parameters())

        # check if both model contains the same set of keys
        assert model_params.keys() == shadow_params.keys()

        for name, param in model_params.items():
            # see https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
            # shadow_variable -= (1 - decay) * (shadow_variable - variable)
            shadow_params[name].sub_((1. - self.decay) * (shadow_params[name] - param))

        model_buffers = OrderedDict(self.model.named_buffers())
        shadow_buffers = OrderedDict(self.shadow.named_buffers())

        # check if both model contains the same set of keys
        assert model_buffers.keys() == shadow_buffers.keys()

        for name, buffer in model_buffers.items():
            # buffers are copied
            shadow_buffers[name].copy_(buffer)

    def forward(self, inputs: Tensor, return_feature: bool = False) -> Tensor:
        if self.training:
            return self.model(inputs, return_feature)
        else:
            return self.shadow(inputs, return_feature)

##<font color=blue>***计算参数和flops***

In [44]:
from thop import profile
from thop import clever_format

def count_your_model(model, x, y):
  outs=model(x)
  
def get_pf(model):


  device=torch.device('cuda:0')
  input = torch.randn(1, 3, 224, 224)
  input=input.to(device)
  flops, params = profile(model, inputs=(input, ))
  flops, params = clever_format([flops, params], "%.3f")
  return flops,params

#<font color=red>***自定义训练过程***</font>

##<font color=blue>普通网络

In [45]:
from tqdm import tqdm
import logging
def do_train1(cfg,model,train_loader,val_loader,optimizer,scheduler,criterion,num_query,logger,checkpoint):
  device=torch.device('cuda:0')
  model=model.to(device)
  flops,params=get_pf(model)
  logger.info('param:{}'.format(params))
  logger.info('flops:{}'.format(flops))
  epochs=1
  if cfg['ema']:
    model=EMA(model,0.999)
  if cfg['amp']:
    scaler=GradScaler()
  with tqdm(total=epochs) as pbar:
    for epoch in range(epochs):
      model.train()
      
      for images,pids in train_loader:
        loss=0
        optimizer.zero_grad()
        images=images.to(device)
        if cfg['amp']:
          if cfg['grad_l2']:
            with autocast():
              outs = model(images)
              pids=pids.to(device)
              for cri,weight in criterion:
                loss=loss+weight*cri(outs,pids)
            new_parameters=[p for p in model.parameters() if p.requires_grad]
            scaled_grad_params = torch.autograd.grad(outputs=scaler.scale(loss),
                                                 inputs=new_parameters,
                                                 create_graph=True)
            inv_scale = 1./scaler.get_scale()
            grad_params = [p * inv_scale for p in scaled_grad_params]
                                     
            with autocast():
              grad_norm = 0
              for grad in grad_params:
                  grad_norm += grad.pow(2).sum()
              grad_norm = grad_norm.sqrt()
              loss = loss + grad_norm  
            scaler.scale(loss).backward() 
            scaler.step(optimizer)
            scaler.update()                                  

          else:
            with autocast():
              outs = model(images)
              pids=pids.to(device)
              for cri,weight in criterion:
                loss=loss+weight*cri(outs,pids)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()  

          if cfg['ema']:
            model.update()
        else:
          outs = model(images)
          pids=pids.to(device)
          for cri,weight in criterion:
            loss=loss+weight*cri(outs,pids)
          loss.backward()
          optimizer.step()
          if cfg['ema']:
            model.update()
      scheduler.step()
      print('eval')
      if epoch%5==0:
        map0,map1=do_eva1(cfg,model,val_loader,num_query)
        print(map0,map1)
        logger.info('epoch:{}---map0:{}---map1:{}----'.format(epoch,map0,map1))
        checkpoint((map0+map1)/2,model,optimizer)
      pbar.update(1)
  
  


# <font color=red>***训练***</font>

##<font color=blue>***普通训练*** 

In [46]:
from torchtools.optim import RangerLars
def main1(cfg,root):
  dataset=PDataset(root)
  train_loader,val_loader,num_query,num_classes=make_date_data_loader(dataset,cfg)

  
  if cfg['model']=='baseline':
      model=Baseline(num_classes,cfg)
  elif cfg['model']=='Global_Attention':
      model=ResNet50_RGA_Model(num_classes=num_classes)
  elif cfg['model']=='Relation':
    model=RelationModel(cfg,6,num_classes)

  
  criterion=make_loss(cfg,num_classes)
  if cfg['optimizer']=='adam':
    optimizer=torch.optim.Adam(model.parameters(),lr=cfg['lr'],weight_decay=cfg['weight_decay'])
  elif cfg['optimizer']=='sgd':
    optimizer=torch.optim.SGD(model.parameters(),lr=cfg['lr'],momentum=cfg['momentum'],weight_decay=cfg['weight_decay'])
  elif cfg['optimizer']=='rangerlars':
    optimizer=RangerLars(model.parameters(),lr=cfg['lr'])
  

  if cfg['scheduler']=='warmup':

    scheduler = WarmupMultiStepLR(optimizer,cfg['steps'], cfg['gamma'],
                  cfg['warmup_factors'],cfg['warmup_iters'], 
                  'linear')
  elif cfg['scheduler']=='flat':
    scheduler=Flat(optimizer,cfg)
  
  elif cfg['scheduler']=='cos_warmup':
    scheduler=torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer ,T_0=5 ,T_mult=2)

  ckpt=cfg['ckpt']
  logpt=cfg['logpt']
  logger=logging.getLogger()
  fh = logging.FileHandler("spam.log")
  fh.setLevel(logging.DEBUG)
  logger.addHandler(fh)
  checkpoint=Checkpoint(ckpt)
  do_train1(cfg,model,train_loader,val_loader,optimizer,scheduler,criterion,num_query,
            logger,checkpoint)


To use this log_lamb_rs, please run 'pip install tensorboardx'. Also you must have Tensorboard running to see results


##<font color=blue>***两阶段***</font>

In [47]:
def main2(cfg,root):
  dataset=Market1501(root)
  train_loader,val_loader,num_query,num_classes=make_data_loader(dataset,cfg)


  model=Baseline(num_classes,cfg)


  criterion=make_loss(cfg,num_classes)

  step1_sgd=torch.optim.SGD(model.parameters(),lr=0.1,weight_decay=0.0001)
  scheduler=Unchange_sch()
  ckpt=cfg['ckpt']
  logpt=cfg['logpt']
  logger=logging.getLogger()
  fh = logging.FileHandler("spam.log")
  fh.setLevel(logging.DEBUG)
  logger.addHandler(fh)
  checkpoint=Checkpoint(ckpt)

  do_train1(cfg,model,train_loader,val_loader,step1_sgd,scheduler,criterion,num_query,
            logger,checkpoint)
  
  state=torch.load(cfg['ckpt'])
  model.load_state_dict(state['model'])
  step2_sgd=torch.optim.SGD(model.parameters(),lr=0.0001,momentum=0.95,weight_decay=0.0001)
  scheduler=torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(step2_sgd,T_0=5,T_mult=2)
  do_train1(cfg,model,train_loader,val_loader,step2_sgd,scheduler,criterion,num_query,
            logger,checkpoint)




#<font color=red>***代码运行***

In [48]:
file1=open('/content/drive/MyDrive/dataset/REID/image_date.json','r')
image_dates=json.load(file1)
file1.close()

file1=open('/content/drive/MyDrive/dataset/REID/image_cam.json','r')
image_cams=json.load(file1)
file1.close()

file1=open('/content/drive/MyDrive/dataset/REID/image_cam_dict.json','r')
image_cam_dict=json.load(file1)
file1.close()

file1=open('/content/drive/MyDrive/dataset/REID/image_p.json','r')
image_p=json.load(file1)
file1.close()

file1=open('/content/drive/MyDrive/dataset/REID/date_id.json','r')
date_ids=json.load(file1)
file1.close()
root='/kaggle/input/p-final-reid/reid1'
root1='/content/datasets/reid1'
main1(cfg,root1)

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torchvision.models.resnet.Bottleneck'>. Treat it as zero Macs and zero Params.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[INFO] Register count_bn() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class '__main__.Baseline'>. Treat it as zero Macs and zero Params.


  0%|          | 0/1 [00:00<?, ?it/s]

eval


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:186: UserWarning: This overload of addmm_ is deprecated:
	addmm_(Number beta, Number alpha, Tensor mat1, Tensor mat2)
Consider using one of the following signatures instead:
	addmm_(Tensor mat1, Tensor mat2, *, Number beta, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:88: RuntimeWarning: invalid value encountered in true_divide


all_ap=0
all_ap=0
all_ap=0
0.39573557863737485 0.07226504510839239


100%|██████████| 1/1 [02:21<00:00, 141.92s/it]
